# NetCDF and CF: The Basics

**DELETE** --> 
This workshop will teach some of the basics of CF metadata for netCDF data files with some hands-on work available in Jupyter Notebooks using Python. Along with introduction to netCDF and CF, we will introduce the CF data model and discuss some netCDF implementation details to consider when deciding how to write data with CF and netCDF. We will cover gridded data as well as in situ data (stations, soundings, etc.) and touch on storing geometries data in CF.

Assume: Basic understanding of netCDF and CF (what they are and how they work together?)

Target Audience: Data producer or manager
- Have data they want to (or have been told they should) write to CF compliant netCDF files

<-- **DELETE**

## Overview of netCDF and CF



### netCDF Data Model

### CF Data Model

## netCDF coordinate variables


## Units

- udunits
- Units for spatial/temporal coordinate variables
  - lat, lon, z
  - time


##- Gridded data
- lat/lon 1-D coordinate variables
- x/y 1-D coordinate variables (with 2-D lat/lon)
  - "coordinates" attribute
  - Grid Mapping
- Cell bounds (and cell measures?)
  - an example
- Cell methods
  - an example
- ??"climatology" attribute??s


## Observational data (CF DSG)
- Overview of Point, station, sounding, trajectory
- Examples
- Reference to [NOAA NCEI netCDF Templates](https://www.nodc.noaa.gov/data/formats/netcdf/v2.0/)


## Geometries

- Overview
- Example
- Link to cfgeom python package


## CF Standard Names
- Overview:
  - Controlled vocabulary of names that describe physical quantity
    - (name, description, canonical units)
  - Canonical units (units used must be convertable to)
  - Help users to decide if data from two sources are comparable
    

## Other topics
- Missing data
  - _FillValue, missing_data, valid_*
- Packed data (scale_factor/add_offset)
- Chunking and compression
- CRS, shape of earth, horizontal and vertial datum


## Recommendations from NASA 
- [NASA Dataset Interoperability Recommendations for Earth Science](https://earthdata.nasa.gov/esdis/eso/standards-and-references/dataset-interoperability-recommendations-for-earth-science) - two PDF documents.
- NASA Data Prodcut Developer's Guide (from the [previous session](https://2019esipsummermeeting.sched.com/event/PtOg/data-product-developers-guide-workshop))
